References:
https://github.com/huggingface/notebooks/blob/master/sagemaker/08_distributed_summarization_bart_t5/sagemaker-notebook.ipynb

In [2]:
!pip install sagemaker transformers --upgrade

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 447 kB 27.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 55.8 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 67.4 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 608 kB/s  eta 0:00:01
     |████████████████████████████████| 3.3 MB 58.3 MB/s eta 0:00:01
     |████████████████████████████████| 748 kB 68.3 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 661 kB/s  eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.63.0-py2.py3-none-any.whl size=622844 sha256=4ea00b63540ea5

In [4]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f"IAM role arn used for running training: {role}")
print(f"S3 bucket used for storing artifacts: {sess.default_bucket()}")

IAM role arn used for running training: arn:aws:iam::847651797386:role/service-role/AmazonSageMaker-ExecutionRole-20211014T151986
S3 bucket used for storing artifacts: sagemaker-eu-west-1-847651797386


In [5]:
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.6.1'}

## Configure distributed training and hyperparameters


In [6]:
hyperparameters={'per_device_train_batch_size': 4,
                 'per_device_eval_batch_size': 4,
                 'model_name_or_path': 'facebook/bart-large-cnn',
                 'do_train': True,
                 'do_eval': True,
                 'do_predict': True,
                 'train_file': 'data/data_train.jsonl',
                 'validation_file': 'data/data_val.jsonl',
                 'test_file': 'data/data_test.jsonl',
                 'text_column': 'text',
                 'summary_column': 'summary',
                 'predict_with_generate': True,
                 'output_dir': '/opt/ml/model',
                 'overwrite_output_dir': True,
                 'num_train_epochs': 3,
                 'learning_rate': 5e-5,
                 'seed': 7,
                 'fp16': True,
                 }

# configuration for running training on smdistributed Data Parallel
# distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

## Create a HuggingFace estimator and start training

In [7]:
from sagemaker.huggingface import HuggingFace

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point='run_summarization.py', # script
    source_dir='./examples/pytorch/summarization', # relative path to example
    git_config=git_config,
    instance_type='ml.p2.xlarge',
    instance_count=1,
    transformers_version='4.6',
    pytorch_version='1.7',
    py_version='py36',
    role=role,
    hyperparameters = hyperparameters,
    # distribution = distribution
)

In [ ]:
# starting the train job
huggingface_estimator.fit()

2021-10-14 14:35:31 Starting - Starting the training job...
2021-10-14 14:35:53 Starting - Launching requested ML instancesProfilerReport-1634222122: InProgress
......
2021-10-14 14:36:59 Starting - Preparing the instances for training............